# Practice similarity functions with Sample Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(2021)

# 1. Euclidean similarity
- 유사도를 계산할 유저를 인덱스로 아이템을 컬럼으로 하는 데이터를 정의
- 유저가 평가하지 않은 아이템에 대해서는 결측값으로 표시

### 1.1 Sample Data

In [2]:
data = [
    [1., None, 1., None],
    [None, 1., 1., None],
    [1., None, 1., 1.],
]

df = pd.DataFrame(
    data=data,
    index=["userA", "userB", "userC"],
    columns=["itemA", "itemB", "itemC", "itemD"],
)

df

,itemA,itemB,itemC,itemD
userA,1.0,NaN,1.0,NaN
userB,NaN,1.0,1.0,NaN
userC,1.0,NaN,1.0,1.0


### 1.2 Remove missing values
- 결측값 0으로 대체

In [3]:
df = df.fillna(0)
df

,itemA,itemB,itemC,itemD
userA,1.0,0.0,1.0,0.0
userB,0.0,1.0,1.0,0.0
userC,1.0,0.0,1.0,1.0


### 1.3 Euclidean similarity calculations

유저-아이템 평가 행렬에서 유저별로 유클리디안 유사도를 계산
$$
유클리디안 유사도 = \frac{1}{\text{유클리디안 거리} + \text{1e-5}}
$$

`sklearn.metrics.pairwise`의 `euclidean_distances`를 이용해 유클리디안 거리를 계산

In [4]:
from sklearn.metrics.pairwise import euclidean_distances

euclidean_distances(
    X=df.loc[["userA"]], 
    Y=df.loc[["userB"]],
)

array([[1.41421356]])

`euclidean_distances`에 X와 Y를 입력할 경우, X와 Y의 각 Row끼리 유클리디안 거리를 계산  
유저A와 유저B의 Row를 각각 X와 Y로 입력하면 두 유저의 유클리디안 거리를 계산

In [5]:
euclidean_distances(df)

array([[0.        , 1.41421356, 1.        ],
       [1.41421356, 0.        , 1.73205081],
       [1.        , 1.73205081, 0.        ]])

유클리디안 거리에 역수를 취해 유클리디안 유사도를 계산

In [6]:
distance = euclidean_distances(df)
similarity = 1 / (distance + 1e-5)
similarity

array([[1.00000000e+05, 7.07101781e-01, 9.99990000e-01],
       [7.07101781e-01, 1.00000000e+05, 5.77346936e-01],
       [9.99990000e-01, 5.77346936e-01, 1.00000000e+05]])

# 2. Cosine similarity

### 2.1 Calculation of cosine similarity
- 유클리디안 유사도 계산에 사용한 데이터를 이용해 코사인 유사도를 계산  
- `sklearn.metrics.pairwise`의 `cosine_similarity`를 이용해 계산
- 유클리디안 유사도 계산과 마찬가지로 X와 Y를 각각 입력할 수 있고, X만 입력할 수 있음

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(
    X=df.loc[["userA"]], 
    Y=df.loc[["userB"]],
)

array([[0.5]])

In [8]:
# 모든 유저 사이의 코사인 유사도 계산
cosine_similarity(df)

array([[1.        , 0.5       , 0.81649658],
       [0.5       , 1.        , 0.40824829],
       [0.81649658, 0.40824829, 1.        ]])

# 3. Pearson similarity
- 아이템에 대한 유저의 선호도를 반영한 행렬에 대해 피어슨 유사도를 적용

### 3.1 Sample Data

In [9]:
# 새로운 데이터 정의
data = [
    [4., 5., 4., 3.],
    [3., 4., 3., 2.],
    [4., 4., 5., 3.],
]

df = pd.DataFrame(
    data=data,
    index=["userA", "userB", "userC"],
    columns=["itemA", "itemB", "itemC", "itemD"],
)

df

,itemA,itemB,itemC,itemD
userA,4.0,5.0,4.0,3.0
userB,3.0,4.0,3.0,2.0
userC,4.0,4.0,5.0,3.0


### 3.2 Pearson similarity calculation
- 피어슨 유사도는 `numpy.corrcoef`를 이용해 계산
- `numpy.corrcoef`는 데이터의 각 Row별로 유사도를 계산

In [10]:
np.corrcoef(df)

array([[1. , 1. , 0.5],
       [1. , 1. , 0.5],
       [0.5, 0.5, 1. ]])

### 3.3 Calculation of cosine similarity
- 피어슨 유사도는 유저 또는 아이템 별로 특성을 제거한 데이터에 코사인 유사도를 적용한 것과 동일
- 유저별로 선호도 평균을 계산하고, 기존 데이터에서 유저별 선호도를 제거
    1. `df.mean(axis=1)`은 각 행에 대해 평균을 계산
    2. `df1.sub(df2, axis=0)`은 인덱스를 기준으로 두 데이터의 차를 계산

In [11]:
 df.mean(axis=1)

userA    4.0
userB    3.0
userC    4.0
dtype: float64

In [12]:
user_mean = df.mean(axis=1)
df_sub = df.sub(user_mean, axis=0)

df_sub

,itemA,itemB,itemC,itemD
userA,0.0,1.0,0.0,-1.0
userB,0.0,1.0,0.0,-1.0
userC,0.0,0.0,1.0,-1.0


In [13]:
cosine_similarity(df_sub)

array([[1. , 1. , 0.5],
       [1. , 1. , 0.5],
       [0.5, 0.5, 1. ]])

## 4. Jacquard similarity

### 4.1 Sample Data
- 유저마다 다른 아이템에 대해 선호도를 평가한 데이터를 정의

In [14]:
data = [
    [4., 0., 4., 3., 0.],
    [3., 4., 0., 2., 0.],
    [0., 0., 4., 5., 3.],
]

df = pd.DataFrame(
    data=data,
    index=["userA", "userB", "userC"],
    columns=["itemA", "itemB", "itemC", "itemD", "itemE"],
)

df

,itemA,itemB,itemC,itemD,itemE
userA,4.0,0.0,4.0,3.0,0.0
userB,3.0,4.0,0.0,2.0,0.0
userC,0.0,0.0,4.0,5.0,3.0


### 4.2 Calculating Jacquard Similarity
- `sklearn.metrics`의 `jaccard_score`를 이용해 자카드 유사도를 계산
- `jaccard_score`는 값의 크기는 무시하고 아이템의 유무를 0과 1로 표현
- 0보다 큰 값을 가지는 경우 선호도를 평가한 것을 표현하기 위해 1로 대체

In [15]:
from sklearn.metrics import jaccard_score

df[df > 0] = 1
df

,itemA,itemB,itemC,itemD,itemE
userA,1.0,0.0,1.0,1.0,0.0
userB,1.0,1.0,0.0,1.0,0.0
userC,0.0,0.0,1.0,1.0,1.0


`jaccard_score`는 비교하는 두 유저의 값을 각각 입력

In [16]:
jaccard_score(
    df.loc["userB"],
    df.loc["userC"],
)

0.2

# 5. Summary

> 추천 시스템
- 사용자의 취향을 알아내 새로운 아이템 추천하는 것

>> 추천 시스템 알고리즘
- Contents Based Filtering : 유저가 기존에 선호한 아이템과 유사 아이템을 추천
    - TF-IDF(Term Frequency ‒ Inverse Document Frequency)
        - 텍스트 기반 컨텐츠의 특징 벡터 추출 방법
        - 출현 빈도를 이용해 특성 문서 내에서 키워드의 중요도 측정
        - 자주 등장하는 키워드는 낮은 중요도 부여
- Collaborative Filtering(협업 필터링) : 여러 유저의 과거 아이템 상호작용 정보를 이용해 추천
    - 메모리 기반(Memory Based) 협업 필터링 : 여러 유저의 과거 아이템 상호작용 정보를 메모리에 저장하고, 추천 필요할 때마다 전체 데이터 이용해서 추천
        - 사용자 기반(User Based) : 아이템 선호도가 비슷한 유저를 탐색하고 비슷한 유저가 좋아한 것 중 새로운 아이템 추천
        - 아이템 기반(Item Based) : 유저들의 선호도가 비슷한 아이템 탐색하고, 유저가 기존에 선호한 아이템과 유사한 아이템 추천
    - 모델 기반(Model Based) : 여러 유저의 과거 아이템 상호작용 정보를 이용해 추천 위한 모델 흑삽하고, 학습된 모델을 이용해 추천
        - 잠재요인 기반(Latent Factor Model) : 관찰된 값을 설명하는 잠재된 특정(요인)들을 밝히고, 이를 이용해 추천
            - 행렬분해(Matrix Factorization)
                - 유저 - 아이템 행렬을 유저와 아이템 행렬로 분해하는 방법
                - 유저가 평가하지 않은 아이템에 대한 선호도를 예측 가능
    - KNN 협업 필터링 : 가장 유사한 K개의 이웃을 통해 아이템을 추천하는 방법, 유저별 아이템 선호도 예측
        - KNN Basic
            - KNN Basic 유저 기반 : 아이템 i에 대한 유전 u의 선호도를 예측
                - 1. 유저 간의 유사도 계산
                - 2. 아이템 i를 평가한 유저들 중에서 유저 u와 비슷한 유저 k명을 찾는다
                - 3. k명의 유사한 유저들이 아이템 i에 평가한 선호도를 유사도 기준으로 가중 평균한다
                - 4. 예측 선호도가 높은 아이템을 유저에게 추천
            - KNN Basic 아이템 기반 : 아이템 i에 대한 유전 u의 선호도를 예측
                - 1. 아이템 간의 유사도 계산
                - 2. 아이템 i와 비슷한 아이템을 k개 찾는다
                - 3. 유저가 평가한 k개의 아이템의 선호드를 유사도 기준으로 가중 평균한다
                - 4. 예측 선호도가 높은 아이템을 유저에게 추천
        - KNN with Means : 선호도의 평균에 선호도 편차를 유사도 기준으로 가중 평균 더하는 방법, 유저/아이템의 평균 선호도 반영
            - KNN with Means 유저 기반
                - 1. 유저 간의 유사도 계산
                - 2. 아이템 i를 평가한 유저들 중에서 유저 u와 비슷한 유저 k명을 찾는다
                - 3. 아이템 i의 평균 선호도 계산
                - 4. k명의 유사한 유저들이 아이템 i에 평가한 선호도의 편차를 유사도 기준으로 가중 평균한다
                - 5. 예측 선호도가 높은 아이템을 유저에게 추천
            - KNN with Means 아이템 기반
                - 1. 아이템간의 유사도를 계산
                - 2. 아이템 i와 비슷한 아이템을 k개 찾는다
                - 3. 아이템 i의 평균 선호도 계산
                - 4. 유저가 평가한 k개의 아이템의 선호도의 편차를 유사도 기준 가중 평균한다
                - 5. 예측 선호도가 높은 아이템을 유저에게 추천
            
> 유사도 함수
- 유사도 : 비슷한 정도를 나타내는 지표
- 거리 기반 유사도 : 좌표 기준 가까운 위치에 있는 점들의 유사도가 높아지도록 측정
    - ex) 유클리디안 유사도
- 각도 기반 유사도 : 좌표 기준으로 방향이 비슷한 점들의 유사도가 높아지도록 측정
    - ex) 코사인 유사도, 피어슨 유사도, 자카드 유사도
    
>> 유사도 함수 종류
- 유클리디안 유사도
    - 유클리디안 거리의 역수로 정의
- 코사인 유사도
    - 두 벡터 간의 코사인 각도를 이용해 계산
    - 1~1 사이 값으로 계산되며, 방향 같은 경우 1, 반대 경우 -1 값을 갖음
- 피어슨 유사도
    - 두 벡터의 상관 계수로 정의
    - 유저 또는 아이템별로 가지는 특성 제거해 점수가 편향된 경우 효과적
- 자카드 유사도
    - 유저가 상호작용한 아이템의 합집합과 교집합의 비율로 계산
    - 유저가 아이템에 평가한 점수의 크기가 무시됨